# Import Libraries

In [12]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
import os
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()

True

# LLM Model

In [13]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [14]:
# testing
llm.invoke("what is your name")

AIMessage(content="I am a large language model, trained by Google.  I don't have a name.\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-fcf1299a-17b3-4c67-9de6-387dea711e96-0', usage_metadata={'input_tokens': 5, 'output_tokens': 21, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}})

# Retriever

In [15]:
model_name = "all-MiniLM-L6-v2"
embeddings = SentenceTransformerEmbeddings(model_name=model_name)
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="../chroma_langchain_db",
)
vector_store.similarity_search("when to eat salad",k=3)

[Document(metadata={'head': 'Healthy Recipes', 'source': 'https://www.everydayhealth.com/diet-nutrition/meal-prep-tips-every-beginner-should-know/', 'title': '_Meal_Prep_Tips_Every_Beginner_Should_Know'}, page_content='1. Make a Plan and Prep ScheduleOne of the best ways to meal prep is to set aside an hour or two every week to draft a meal plan and build your shopping list. Mia Syn, MS, RDN, the owner of\xa0Nutrition by Mia\xa0in Charleston, South Carolina, recommends penciling it into your calendar and treating it as a priority — just as you would an important date or work meeting.Additionally, Palinski-Wade suggests allocating separate time for post-shopping prep — whether that’s simply cutting vegetables into snack-ready slices, or cooking whole batch-style freezer meals. Find what works for you and stick to it, and over time you’ll create a good habit that’s second nature.'),
 Document(metadata={'head': 'Healthy Recipes', 'source': 'https://www.everydayhealth.com/diet-nutrition/me

In [16]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [17]:
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# History Aware Chat Prompt Template

In [18]:
contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [19]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []

In [20]:
system_prompt = (
    "You are an health doctor A.I who provide medical advices for queries. "
    "communicate as a real doctor"
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use five sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [21]:
rag_chain.invoke({"input":"symptoms of brest cancer?","chat_history":chat_history,"context":history_aware_retriever})

{'input': 'symptoms of brest cancer?',
 'chat_history': [],
 'context': [Document(metadata={'head': '', 'source': 'https://www.everydayhealth.com/breast-cancer/metastatic-breast-cancer/guide/', 'title': 'What_Is_Metastatic_Breast_Cancer_Symptoms_Causes_Treatments_and_More'}, page_content='Symptoms of breast cancer that has spread to the bones include:Bones are easily broken or fracturedBone painSwellingSymptoms of breast cancer that has spread to the brain include:Changes in behaviorIssues with memoryNausea or vomitingPersistent headaches\xa0SeizuresSpeech or vision issuesSymptoms of breast cancer that has spread to the liver include:Abdominal painJaundice (yellowing of the skin or whites of the eyes)Nausea or vomitingRash or itchy skinHigh liver enzymes in the bloodSymptoms of breast cancer that has spread to the lungs include:Pain in the chestPersistent coughShortness of breath'),
  Document(metadata={'head': '', 'source': 'https://www.everydayhealth.com/breast-cancer/what-is-triple-

In [22]:
from langchain_core.messages import HumanMessage, AIMessage
while True:
    question = input()
    answer = rag_chain.invoke({"input": question, "chat_history": chat_history})['answer']
    chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=answer)
    ])
    
    print(f"Human: {question}")
    print(f"AI: {answer}\n")

Human: hey doctor
AI: Hello!  How can I help you today? Please tell me about any symptoms or concerns you have.  I can offer information and advice, but remember I can't provide a diagnosis or prescribe medication.  It's important to see a doctor in person for personalized care.




KeyboardInterrupt: Interrupted by user

In [ ]:
import sqlite3
from datetime import datetime
import uuid

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    user_query TEXT,
    gpt_response TEXT,
    model TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()